In [11]:
import os
import numpy as np
from TCFile import TCFile

root_dir = r"C:\rkka_Projects\cell_death_v1\Data\Hela_Segmentation_test_PWS\TCF"

tcf_list = []

pathways = dict()
pathways['necroptosis_list'] = []
pathways['apoptosis_list'] = []
pathways['necrosis_list'] = []
pathways['live_list'] = []


for p in os.listdir(root_dir):
    if p.endswith('.TCF'):
        tcf_list.append(p)
        if 'Hoechst' in p:
            pathways['necroptosis_list'].append(p)
        elif 'CD95' in p:
            pathways['apoptosis_list'].append(p)
        elif 'none' in p:
            pathways['live_list'].append(p)
        else:
            pathways['necrosis_list'].append(p)

In [12]:
ps = ['apoptosis', 'necroptosis', 'necrosis', 'live']
file_list = []
for p in ps:
    for i in pathways[p+'_list']:
        file_list.append(os.path.join(root_dir, i))

time_list = [34, 35, 36]


In [6]:
ps = ['apoptosis', 'necroptosis', 'necrosis', 'live']
file_list = []
for p in ps:
    for i in pathways[p+'_list']:
        file_list.append(os.path.join(root_dir, i))

time_list = [34, 35, 36]

min_list = []
max_list = []

for idx, path in enumerate(file_list):
    path = file_list[idx]
    file = TCFile(path, '3D')
    for time in time_list:
        min, max = np.min(np.max(file[time], axis=0)*file.data_resolution[0]), np.max(np.max(file[time], axis=0)*file.data_resolution[0])
        min_list.append(min)
        max_list.append(max)

global_min, global_max = np.min(np.array(min_list)), np.max(np.array(max_list))
global_min, global_max

c:\miniconda3\envs\cell\Lib\site-packages\TCFile\TCFile_class.py:190: UserWarning: You use an experimental file format deprecated.
Update your reconstruction program and rebuild TCF file.
  warnings.warn(("You use an experimental file format deprecated.\n"


(1.0740192, 1.528778)

<h1> 2D </h1>

In [ ]:
from PIL import Image
from tqdm import tqdm
import cv2
import utils
import torch
from torchvision import models

# configurations
times = [34, 35, 36]
mode = 'qpi'
crop_size = (480, 480)
save_path = 'C:/rkka_Projects/cell_death_v1/Data/Hela_Segmentation_test_PWS/TCF/pathway_train_qpi'
root_dir = 'C:/rkka_Projects/cell_death_v1/Data/Hela_Segmentation_test_PWS/TCF/'

# Crop
for pathway in tqdm(list(pathways.keys())):
    

    for file_path in pathways[pathway]:    
        file = TCFile(os.path.join(root_dir, file_path), '3D')
        for time_index in times:
            image_3d = file[time_index]
            fz = file.data_resolution[0]/0.1632  
            fy = file.data_resolution[1]/0.1632
            fx = file.data_resolution[1]/0.1632

            image_3d = torch.from_numpy(image_3d)
            image_3d = torch.nn.functional.interpolate(image_3d, scale_factor=(fz, fy, fx))
            image_3d = np.array(image_3d)
            slice_2d = np.sum(image_3d, axis=0)
            
            if fx!=1 and fy!=1:
                slice_2d = cv2.resize(slice_2d, dsize=(1+int(slice_2d.shape[0]*fy), 1+int(slice_2d.shape[0]*fx)))

            patches = utils.crop_patch(slice_2d, crop_size=crop_size)
            for patch_index, patch in enumerate(patches):
                top = patch[0]
                left = patch[1]
                temp = slice_2d[top:top+crop_size[0], left:left+crop_size[1]]
                temp = utils.image_normalization(temp)
                image = Image.fromarray(temp)
                image.save(f"{save_path}/{pathway[:-5]}/file_{file_path[:-4]}_patch_{patch_index}_t_{time_index}.png")

print('finished!')

<h1> 3D </h1>

In [2]:
import os
import numpy as np
from TCFile import TCFile

root_dir = r"C:\rkka_Projects\cell_death_v1\Data\Hela_Segmentation_test_PWS\TCF\test"

tcf_list = []

pathways = dict()
pathways['necroptosis_list'] = []
pathways['apoptosis_list'] = []
pathways['necrosis_list'] = []
pathways['live_list'] = []


for p in os.listdir(root_dir):
    if p.endswith('.TCF'):
        tcf_list.append(p)
        if 'Hoechst' in p:
            pathways['necroptosis_list'].append(p)
        elif 'CD95' in p:
            pathways['apoptosis_list'].append(p)
        elif 'none' in p:
            pathways['live_list'].append(p)
        else:
            pathways['necrosis_list'].append(p)

In [4]:
from PIL import Image
from tqdm import tqdm
import tifffile
import utils
import torch

# configurations
times = [34, 35, 36]
mode = 'mip'
z_range_ratio = 40/60
crop_size = (480, 480)
save_path = 'C:/rkka_Projects/cell_death_v1/Data/Hela_Segmentation_test_PWS/TCF/test/pathway_test_3d'
root_dir = 'C:/rkka_Projects/cell_death_v1/Data/Hela_Segmentation_test_PWS/TCF/test/'

# Crop
for pathway in tqdm(list(pathways.keys())):
    for file_path in pathways[pathway]:
        file = TCFile(os.path.join(root_dir, file_path), '3D')
        for time_index in times:
            
            file_array = file[time_index]
            tensor = torch.tensor(file_array).unsqueeze(0).unsqueeze(0).cuda()
            
            fz = file.data_resolution[0]/0.1632
            fy = file.data_resolution[1]/0.1632
            fx = file.data_resolution[2]/0.1632
            
            resampled_tensor = torch.nn.functional.interpolate(
                tensor, scale_factor=(fz, fy, fx), mode='trilinear', align_corners=False
            )
 
            resampled_tensor = resampled_tensor.squeeze(0).squeeze(0)
            file_array = np.array(resampled_tensor.cpu())
            z_range = int(z_range_ratio * resampled_tensor.shape[0])
            
            patches = utils.crop_patch(file_array[0], crop_size=crop_size)
            for patch_index, patch in enumerate(patches):
                z_center = file_array.shape[0]//2
                top = patch[0]
                left = patch[1]
                temp = file_array[z_center-z_range//2:z_center+z_range//2, top:top+crop_size[0], left:left+crop_size[1]]
                tifffile.imwrite(f"{save_path}/{pathway[:-5]}/file_{file_path[:-4]}_patch_{patch_index}_t_{time_index}.tif", temp)
                
print('finished!')

100%|██████████| 4/4 [03:42<00:00, 55.57s/it]

finished!


In [47]:
import napari
viewer = napari.Viewer()

In [7]:
viewer.add_image(np.array(file))

<Image layer 'Image' at 0x15f981535f0>

In [9]:
import os

# Directories
folder_2d = r"C:\rkka_Projects\cell_death_v1\Data\Hela_Segmentation_test_PWS\TCF\test\pathway_test_mip\live"  # Replace with the path to your 2D folder
folder_3d = r"C:\rkka_Projects\cell_death_v1\Data\Hela_Segmentation_test_PWS\TCF\test\pathway_test_3d\live"  # Replace with the path to your 3D folder

# List filenames in 2D folder (without extensions)
files_2d = {os.path.splitext(file)[0] for file in os.listdir(folder_2d)}

# List files in 3D folder
files_3d = os.listdir(folder_3d)

# Delete 3D files not in 2D folder
for file in files_3d:
    file_name, ext = os.path.splitext(file)  # Split filename and extension
    if file_name not in files_2d:
        file_path = os.path.join(folder_3d, file)
        os.remove(file_path)  # Delete the file
        print(f"Deleted: {file_path}")

print("Cleanup completed!")

Deleted: C:\rkka_Projects\cell_death_v1\Data\Hela_Segmentation_test_PWS\TCF\test\pathway_test_3d\live\file_241126.155819.hela_none_PP2.001.Group1.A1.T001P08_patch_0_t_34.tif
Deleted: C:\rkka_Projects\cell_death_v1\Data\Hela_Segmentation_test_PWS\TCF\test\pathway_test_3d\live\file_241126.155819.hela_none_PP2.001.Group1.A1.T001P08_patch_0_t_35.tif
Deleted: C:\rkka_Projects\cell_death_v1\Data\Hela_Segmentation_test_PWS\TCF\test\pathway_test_3d\live\file_241126.155819.hela_none_PP2.001.Group1.A1.T001P08_patch_0_t_36.tif
Deleted: C:\rkka_Projects\cell_death_v1\Data\Hela_Segmentation_test_PWS\TCF\test\pathway_test_3d\live\file_241126.155819.hela_none_PP2.001.Group1.A1.T001P08_patch_1_t_34.tif
Deleted: C:\rkka_Projects\cell_death_v1\Data\Hela_Segmentation_test_PWS\TCF\test\pathway_test_3d\live\file_241126.155819.hela_none_PP2.001.Group1.A1.T001P08_patch_1_t_35.tif
Deleted: C:\rkka_Projects\cell_death_v1\Data\Hela_Segmentation_test_PWS\TCF\test\pathway_test_3d\live\file_241126.155819.hela_none

<h1> Large FOV </h1>

In [10]:
from PIL import Image
import cv2
import utils
import os

# configurations
times = [34, 35, 36]
mode = 'mip'
crop_size = (480, 480)
save_path = r"C:\rkka_Projects\cell_death_v1\Data\cell death_B4C4B5_0808\pathway_train_mip"
root_dir = r"C:\rkka_Projects\cell_death_v1\Data\cell death_B4C4B5_0808"

file_list = ["240808.200653.death_B4C4B5.005.Group1.B4.T001P01.TCF",
             "240808.200653.death_B4C4B5.005.Group2.C4.T001P01.TCF",
             "240808.200653.death_B4C4B5.005.Group3.B5.T001P01.TCF"]

# Crop
for file_path in tqdm(file_list):
    file = TCFile(os.path.join(root_dir, file_path), '3D')
    for time_index in times:
        if mode=='qpi':
            slice_2d = np.sum(file[time_index], axis=0) * file.data_resolution[0]
        elif mode=='mip':
            slice_2d = np.max(file[time_index], axis=0)
                
        fy = file.data_resolution[1]/0.1632
        fx = file.data_resolution[1]/0.1632
        
        if fx!=1 and fy!=1:
            slice_2d = cv2.resize(slice_2d, dsize=(1+int(slice_2d.shape[0]*fy), 1+int(slice_2d.shape[0]*fx)))

        patches = utils.crop_patch(slice_2d, crop_size=crop_size)
        for patch_index, patch in enumerate(patches):
            top = patch[0]
            left = patch[1]
            temp = slice_2d[top:top+crop_size[0], left:left+crop_size[1]]
            temp = utils.image_normalization(temp)
            image = Image.fromarray(temp)
            image.save(f"{save_path}/file_{file_path[:-4]}_patch_{patch_index}_t_{time_index}.png")

print('finished!')

100%|██████████| 3/3 [00:51<00:00, 17.26s/it]

finished!
